In [2]:
import numpy as np
from numpy.linalg import pinv, norm
import matplotlib.pyplot as plt

# プロット関数の定義
import os

def Plot(X, Y, iteration):
    directory = 'data/'
    if not os.path.exists(directory):
        os.makedirs(directory)

    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)
    ax.axis([-1, 4, -1, 4])
    ax.grid()
    ax.plot(X, Y)
    ax.scatter(X, Y)
    ax.plot(Target[0], Target[1], marker='x', ms=20, color='red')
    ax.text(X[0], Y[0]-0.4, 'Base', ha='center')
    ax.text(X[-1], Y[-1]+0.2, 'End Effector', ha='left')
    ax.text(Target[0], Target[1]-0.2, 'Target', ha='left', va='top', color='red')
    plt.savefig(f'{directory}plot_{iteration}.png')
    plt.close()

# ヤコビアン関数の定義
# ここに FK, Jacobian 関数の定義を追加
def Jacobian(L, TH):
    # 各関節の位置を計算
    x1 = L[0] * np.cos(TH[0] + np.pi / 2)
    y1 = L[0] * np.sin(TH[0] + np.pi / 2)
    x2 = x1 + L[1] * np.cos(TH[0] + TH[1] + np.pi / 2)
    y2 = y1 + L[1] * np.sin(TH[0] + TH[1] + np.pi / 2)
    x3 = x2 + L[2] * np.cos(TH[0] + TH[1] + TH[2] + np.pi / 2)
    y3 = y2 + L[2] * np.sin(TH[0] + TH[1] + TH[2] + np.pi / 2)

    # ヤコビアン行列の計算
    J = np.array([
        [-L[0]*np.sin(TH[0] + np.pi / 2) - L[1]*np.sin(TH[0] + TH[1] + np.pi / 2) - L[2]*np.sin(TH[0] + TH[1] + TH[2] + np.pi / 2), 
         -L[1]*np.sin(TH[0] + TH[1] + np.pi / 2) - L[2]*np.sin(TH[0] + TH[1] + TH[2] + np.pi / 2), 
         -L[2]*np.sin(TH[0] + TH[1] + TH[2] + np.pi / 2)],
        [ L[0]*np.cos(TH[0] + np.pi / 2) + L[1]*np.cos(TH[0] + TH[1] + np.pi / 2) + L[2]*np.cos(TH[0] + TH[1] + TH[2] + np.pi / 2), 
          L[1]*np.cos(TH[0] + TH[1] + np.pi / 2) + L[2]*np.cos(TH[0] + TH[1] + TH[2] + np.pi / 2), 
          L[2]*np.cos(TH[0] + TH[1] + TH[2] + np.pi / 2)]
    ])
    return J

def FK(L, TH, plot=False):
    # 各関節の角度を計算
    th1 = TH[0] + np.pi / 2
    th2 = TH[1] + TH[0] + np.pi / 2
    th3 = TH[2] + TH[1] + TH[0] + np.pi / 2

    # エンドエフェクタの位置を計算
    x0 = 0
    y0 = 0.7
    x1 = x0 + L[0] * np.cos(th1)
    y1 = y0 + L[0] * np.sin(th1)
    x2 = x1 + L[1] * np.cos(th2)
    y2 = y1 + L[1] * np.sin(th2)
    x3 = x2 + L[2] * np.cos(th3)
    y3 = y2 + L[2] * np.sin(th3)
    if plot:
        return [x0, x1, x2, x3], [y0, y1, y2, y3]
    else:
        return np.array([x3, y3])

# 角度の制限関数
def limit_angles(TH, TH_min, TH_max):
    TH_limited = np.clip(TH, TH_min, TH_max)
    return TH_limited

# パラメータの設定
L = np.array([0.83, 0.935, 1.25])  # リンクの長さ
TH = np.radians([0, 0, 0])  # 初期関節角度
Target = np.array([3, 2])  # 目標位置
TH_max = np.radians([105, 145, 105])  # 上限
TH_min = np.radians([-105, -145, -105])  # 下限
scaler = 0.1
loops = 0

# メインループ
while True:
    J = Jacobian(L, TH)
    Err = Target - FK(L, TH)
    dTheta = pinv(J, rcond=1e-15) @ Err.T

    if dTheta.ndim == 1:
        TH += dTheta * scaler
    else:
        TH += dTheta[:, 0] * scaler

    TH = limit_angles(TH, TH_min, TH_max)
    loops += 1
    X, Y = FK(L, TH, plot=True)
    Plot(X, Y, loops)

    if norm(Err) < 1e-5 or loops > 500:
        break

# 結果


In [3]:
import cv2
import os
import numpy as np

def create_video(image_folder, video_name, fps=10):
    images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
    images.sort(key=lambda x: int(x.split('_')[1].split('.')[0]))  # 'plot_1.png', 'plot_2.png', etc のように並べ替える

    # 最初の画像からフレームの大きさを取得
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    # VideoWriter オブジェクトを作成
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    # 画像を動画に追加
    for image in images:
        video.write(cv2.imread(os.path.join(image_folder, image)))

    cv2.destroyAllWindows()
    video.release()

# 使用例
create_video('data/', 'data/robot_arm_animation.mp4')

error: OpenCV(4.8.0) /io/opencv/modules/highgui/src/window.cpp:1266: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'
